## Merging Files

In [5]:
import pandas as pd
import re

# Function to extract the financial year from the file name
def extract_financial_year(file_name):
    match = re.search(r'(\d{4})', file_name)
    if match:
        return match.group(1)
    else:
        raise ValueError(f"No financial year found in file name: {file_name}")

# File paths (replace with your actual file paths)
file_path_1 = 'apps_by_day_2021.csv'
file_path_2 = 'apps_by_day_2022.csv'

# Read the CSV files
df1 = pd.read_csv(file_path_1)
df2 = pd.read_csv(file_path_2)

# Rename the columns to a common set of names
df1.rename(columns={'date_applied_2021': 'date_applied', 'paid_apps_2021': 'paid_apps', 'all_apps_2021': 'all_apps'}, inplace=True)
df2.rename(columns={'date_applied_2022': 'date_applied', 'paid_apps_2022': 'paid_apps', 'all_apps_2022': 'all_apps'}, inplace=True)

# Describing each datasets before merging
description1 = df1.describe(include='all')
print("Dataset 1 Description:")
print(description1)
description2 = df2.describe(include='all')
print("\nDataset 2 Description:")
print(description2)

# Concatenate the datasets
merged_df = pd.concat([df1, df2], ignore_index=True)

# Save the merged dataset to a new CSV file
merged_file_path = 'merged_dataset.csv'
merged_df.to_csv(merged_file_path, index=False)

print(f"\nMerged dataset has been saved to {merged_file_path}.")


Dataset 1 Description:
       date_applied    paid_apps     all_apps
count           157   157.000000   157.000000
unique          157          NaN          NaN
top       3/08/2021          NaN          NaN
freq              1          NaN          NaN
mean            NaN   391.547771   406.738854
std             NaN   487.809671   503.904704
min             NaN     1.000000     1.000000
25%             NaN    47.000000    52.000000
50%             NaN    90.000000    98.000000
75%             NaN   725.000000   747.000000
max             NaN  2350.000000  2398.000000

Dataset 2 Description:
       date_applied    paid_apps     all_apps
count           154   154.000000   154.000000
unique          154          NaN          NaN
top       2/08/2022          NaN          NaN
freq              1          NaN          NaN
mean            NaN   400.318182   418.064935
std             NaN   482.405312   498.983800
min             NaN     1.000000     1.000000
25%             NaN    59.000000 

## Finding duplicate records

In [6]:
df = merged_df
# Find duplicate records
duplicates = df[df.duplicated(keep=False)]

# Remove duplicates from the original DataFrame
df = df.drop_duplicates()

# Save duplicate records to a CSV file
duplicates.to_csv('duplicate_records.csv', index=False)

# Show the number of duplicate records
num_duplicates = len(duplicates)
print(f"Number of duplicate records found: {num_duplicates}")

print("Duplicate records saved to duplicate_records.csv")

Number of duplicate records found: 0
Duplicate records saved to duplicate_records.csv


## Describing new dataset

In [7]:
df = merged_df
# Describe the dataset
description = df.describe(include='all')
print("Dataset Description:")
print(description)

# Print the columns' data types
print("\nColumns' Data Types:")
print(df.dtypes)

Dataset Description:
       date_applied    paid_apps     all_apps
count           311   311.000000   311.000000
unique          311          NaN          NaN
top       3/08/2021          NaN          NaN
freq              1          NaN          NaN
mean            NaN   395.890675   412.347267
std             NaN   484.378046   500.696819
min             NaN     1.000000     1.000000
25%             NaN    51.500000    57.000000
50%             NaN    98.000000   108.000000
75%             NaN   721.500000   737.500000
max             NaN  2350.000000  2398.000000

Columns' Data Types:
date_applied    object
paid_apps        int64
all_apps         int64
dtype: object


## Assessing date column

In [8]:
date_column_name = 'date_applied'

# Convert the date column to datetime format for easier manipulation
df[date_column_name] = pd.to_datetime(df[date_column_name], errors='coerce', dayfirst=True)

# Find and print the first and last date of the dataset
first_date = df[date_column_name].min()
last_date = df[date_column_name].max()
print(f"\nFirst date in the dataset: {first_date}")
print(f"Last date in the dataset: {last_date}")

# Check and print any false dates in the dataset
false_dates = df[df[date_column_name].isna()]
if not false_dates.empty:
    print("\nFalse dates found in the dataset:")
    print(false_dates)
else:
    print("\nNo false dates found in the dataset.")


First date in the dataset: 2021-08-03 00:00:00
Last date in the dataset: 2023-01-17 00:00:00

No false dates found in the dataset.


## Assessing Apps columns

In [9]:
# Find and print records where paid_apps_2021 > all_apps_2021
invalid_records = df[df['paid_apps'] > df['all_apps']]
if not invalid_records.empty:
    print("\nRecords where paid_apps_2021 is greater than all_apps_2021:")
    print(invalid_records)
    # Save these invalid records to a CSV file and remove them from the dataset
    invalid_records.to_csv('false_records.csv', index=False)
    df = df.drop(invalid_records.index)
    print("\nInvalid records have been saved to false_records.csv and removed from the dataset.")
else:
    print("\nNo records found where paid_apps_2021 is greater than all_apps_2021.")

    
# Save the cleaned and updated dataset to a new CSV file
df.to_csv('cleaned_dataset.csv', index=False)

print("\nCleaned dataset with cumulative columns has been saved to cleaned_dataset.csv.")



Records where paid_apps_2021 is greater than all_apps_2021:
    date_applied  paid_apps  all_apps
124   2021-12-09          4         1
126   2021-12-13          2         1
130   2021-12-17          2         1
133   2021-12-20          2         1
134   2021-12-21          5         2
138   2021-12-27          2         1
142   2022-01-03          3         1
143   2022-01-05        393         1
144   2022-01-06        135         3
145   2022-01-07        111         2
149   2022-01-11         69        67
150   2022-01-12         60        57
155   2022-01-17        103        87
156   2022-01-18         92        44
282   2022-12-08          2         1
286   2022-12-14          2         1
287   2022-12-15          2         1
288   2022-12-16          6         3
292   2022-12-22          3         1
297   2023-01-04        433         1
298   2023-01-05        163         1
299   2023-01-06        124         1
303   2023-01-10         98        74
304   2023-01-11         87